In [1]:
### Activate the packages required
## in case any module is not installed, run this line --> pip install pandas
import pandas as pd   ## pandas is used for working around data
import numpy as np  ## numpy is used for numerical/statistical functions
import matplotlib.pyplot as plt  ## matplotlib is used for creating charts/graphics
import pandas_datareader.data as web  ## used for extracting data from web sources
import yfinance as yf    ## is used for extracting data from yahoo finance
yf.pdr_override()   ## bugfix for conflict between pandas_datareader & yfinance
from datetime import datetime   ## is used to convert the date & time format of the data

In [2]:
### User Inputs
startdate = datetime(2022, 1, 1)   ## from what date the price data to be downloaded
enddate = datetime(2023, 12, 31)   ## till what date the price data to be downloaded
tickers = ['APARINDS.NS', 'OIL.NS', 'APOLLOTYRE.NS', 'TATACHEM.NS', 'NH.NS']   ## ticker symbols of the stocks using which the portfolio needs to be built
number_of_portfolios = 5000  ## I want create x number of randomly weighted portfolios from which the best need to be chosen
risk_free = 0.071  ## risk-free rate

In [3]:
### Create an empty table spcace for storing the data
returns = pd.DataFrame()

In [4]:
### Create a lopp function that downloads each stock's price data & compute returns

for ticker in tickers:
  data = web.get_data_yahoo(ticker, start = startdate, end = enddate)   ## download from yfinance
  data = pd.DataFrame(data)   ## store it in a table form
  data[ticker] = data['Adj Close'].pct_change()  ## computing the returns for each day
  if returns.empty:
    returns = data[[ticker]]
  else:
    returns = returns.join(data[[ticker]], how = 'outer')   ## adds a new column next to existing and store the return data

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [5]:
## drop the missing values
returns = returns.dropna()
returns.head()   ### displays the top 5 rows of a dataframe

,APARINDS.NS,OIL.NS,APOLLOTYRE.NS,TATACHEM.NS,NH.NS
Date,,,,,
2022-01-04,-0.000785,-0.009306,0.021277,-0.005524,-0.008193
2022-01-05,-0.011448,-0.010155,0.008422,0.006710,0.028639
2022-01-06,0.024419,0.014876,0.004396,-0.012074,-0.026312
2022-01-07,0.008140,0.081880,0.019694,0.025494,0.016496
2022-01-10,-0.000384,0.004438,-0.002146,0.012996,0.012751


In [6]:
### Create variables to compute & store data
portfolio_weights = []
portfolio_returns = []
portfolio_risks = []
portfolio_sharpe = []

In [7]:
### Compute the weights, returns, risks & sharpe for number of portfolio defined

for portfolio in range(number_of_portfolios):
  weights = np.random.random_sample(len(tickers))  ## generated 5 random numbers
  weights = np.round(weights/np.sum(weights),3)   ## converts the random numbers into weights
  portfolio_weights.append(weights)   ## add the weights to the file portfolio_weights
  ann_return = np.sum(returns.mean()*weights)*252        ## compute the portfolio return as weighted average
  portfolio_returns.append(ann_return)   ## add the return to the file portfolio_returns
  covar_matrix = returns.cov()*252   ### creates covar matrix
  ann_stdev = np.sqrt(np.dot(weights.T, np.dot(covar_matrix, weights)))   ## computes the portfolio stdev
  portfolio_risks.append(ann_stdev)   ## add the stdev to the file portfolio_risks
  sharpe = (ann_return - risk_free)/ann_stdev  ## computes the sharpe ratio
  portfolio_sharpe.append(sharpe)   ## add the sharpe to the file portfolio_sharpe

portfolio_returns = np.array(portfolio_returns)  ## store the data in array form
portfolio_risks = np.array(portfolio_risks)
portfolio_sharpe = np.array(portfolio_sharpe)

portfolio_parameters = [portfolio_returns, portfolio_risks, portfolio_sharpe, portfolio_weights]
portfolio_data = pd.DataFrame(portfolio_parameters).T
portfolio_data.columns = ['Return', 'Risk', 'Sharpe', 'Weights']
print(portfolio_data)

        Return      Risk    Sharpe                              Weights
0     0.409519  0.191497  1.767749  [0.016, 0.155, 0.393, 0.046, 0.391]
1     0.362416  0.185236  1.573216  [0.029, 0.152, 0.225, 0.283, 0.311]
2     0.681145  0.252937  2.412243  [0.388, 0.104, 0.066, 0.134, 0.309]
3     0.578291  0.207526  2.444466   [0.25, 0.211, 0.135, 0.142, 0.262]
4      0.44887  0.182148  2.074526  [0.085, 0.233, 0.213, 0.125, 0.344]
...        ...       ...       ...                                  ...
4995  0.377754  0.188197  1.629962  [0.025, 0.098, 0.232, 0.158, 0.487]
4996  0.339633   0.20789  1.292188   [0.019, 0.12, 0.354, 0.384, 0.123]
4997  0.450496  0.203971  1.860539  [0.122, 0.329, 0.106, 0.296, 0.147]
4998  0.659428  0.256282  2.296017  [0.368, 0.206, 0.155, 0.246, 0.025]
4999  0.538159   0.21938   2.12945   [0.219, 0.198, 0.292, 0.261, 0.03]

[5000 rows x 4 columns]


In [8]:
## identify the max sharpe ratio portfolio
max_sharpe = portfolio_data.iloc[portfolio_data['Sharpe'].astype(float).idxmax()]
print('Highest Sharpe Portfolio:')
print(max_sharpe)
print(tickers)
print("")

Highest Sharpe Portfolio:
Return                                 0.65376
Risk                                  0.222592
Sharpe                                2.618064
Weights    [0.302, 0.235, 0.192, 0.015, 0.256]
Name: 3640, dtype: object
['APARINDS.NS', 'OIL.NS', 'APOLLOTYRE.NS', 'TATACHEM.NS', 'NH.NS']



In [9]:
portfolio_data.to_csv("Portfolio.csv")